In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mne
import pywt
from torch.utils.data import DataLoader, Dataset
from torch import nn
import torch
import pickle
import os
from datetime import datetime
import sqlite3
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.loggers import TensorBoardLogger

from torchmetrics.functional.classification.accuracy import accuracy


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Training on device: {device}')

Training on device: cuda


In [3]:
# def read_cwt_data(db_path):
#     conn = sqlite3.connect(db_path)
#     cursor = conn.cursor()
#     cursor.execute("SELECT cwt_data FROM wavelet_transforms")
#     data = cursor.fetchall()
#     conn.close()

#     # Deserializacja danych
#     cwt_arrays = [pickle.loads(d[0]) for d in data]
#     return cwt_arrays


# # Odczytanie danych z bazy danych
# cwt_data_list = read_cwt_data("cwt_data.db")


# # Przykładowy odczyt jednego tensora CWT
# print(cwt_data_list[0].shape)

In [7]:
class CWTDataset(Dataset):
    def __init__(self, db_path, sequence_length=4000):
        self.db_path = db_path
        self.sequence_length = sequence_length
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self.cursor.execute("SELECT COUNT(*) FROM wavelet_transforms")
        self.total_samples = self.cursor.fetchone()[0]

    def __len__(self):
        # Aby umożliwić nachodzenie, liczba możliwych sekwencji będzie równa liczbie próbek minus długość sekwencji + 1
        return self.total_samples - self.sequence_length + 1

    def __getitem__(self, idx):
        # Zwraca sekwencję próbek i target z ostatniej próbki
        
        query = (
            "SELECT cwt_data, target FROM wavelet_transforms WHERE id BETWEEN ? AND ?"
        )
        print(query,(idx + 1, idx + self.sequence_length))
        self.cursor.execute(
            query, (idx + 1, idx + self.sequence_length)
        )  # SQLite indeksuje od 1
        rows = self.cursor.fetchall()

        cwt_sequence = np.array([pickle.loads(row[0]) for row in rows])

        # Target ostatniej próbki w sekwencji
        target = rows[-1][1]

        cwt_tensor = torch.tensor(cwt_sequence, dtype=torch.float32)
        target_tensor = torch.tensor(target, dtype=torch.int64)
        return cwt_tensor, target_tensor

    def __del__(self):
        self.conn.close()

In [8]:
from torch.utils.data import DataLoader

# Tworzenie instancji datasetu
cwt_dataset = CWTDataset('cwt_data.db', 4000)

# batch_size = 10  # Liczba sekwencji w jednym batchu
# train_loader = DataLoader(dataset=cwt_dataset, batch_size=batch_size, shuffle=True,num_workers=15)


In [12]:
cwt_dataset.__getitem__(0)[0].shape

SELECT cwt_data, target FROM wavelet_transforms WHERE id BETWEEN ? AND ? (1, 4000)


torch.Size([4000, 64, 10])

In [18]:
cwt_dataset.__getitem__(1)[0].shape

SELECT cwt_data, target FROM wavelet_transforms WHERE id BETWEEN ? AND ? (2, 4001)


torch.Size([4000, 64, 10])

In [17]:
cwt_dataset.__getitem__(2000)[1]

SELECT cwt_data, target FROM wavelet_transforms WHERE id BETWEEN ? AND ? (2001, 6000)


tensor(2)

In [ ]:
# class LSTMModel(nn.Module):
#     def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
#         super(LSTMModel, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.layer_dim = layer_dim
#         self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)
    
#     def forward(self, x):
#         # Inicjalizacja stanów ukrytych
#         h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=x.device)
#         c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=x.device)
        
#         # Forward pass through LSTM layer
#         out, (hn, cn) = self.lstm(x, (h0, c0))
        
#         # Forward pass through linear layer
#         out = self.fc(out[:, -1, :])
#         return out

In [ ]:
# # Parametry
# input_dim = 640
# hidden_dim = 100
# layer_dim = 1
# output_dim = 1
# num_epochs = 20
# learning_rate = 0.01

# model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim).to(device)
# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# # Pętla treningowa
# for epoch in range(num_epochs):
#     model.train()
#     total_loss = 0
#     for batch_idx, (data, targets) in enumerate(train_loader):
#         # Przeniesienie danych na GPU
#         data = data.view(-1, 4000, 640).to(device)
#         targets = targets.to(device)
        
#         optimizer.zero_grad()
#         outputs = model(data)
        
#         loss = criterion(outputs, targets.float().unsqueeze(1))
#         loss.backward()
#         optimizer.step()
        
#         total_loss += loss.item()
#         print(f'Epoch: {epoch+1}, Batch: {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}')
#     avg_train_loss = train_loss / len(train_loader)
#     # Walidacja
#     val_loss = evaluate_model(model, validation_loader, device)  # załóżmy, że mamy validation_loader

#     # Wyświetlanie postępów
#     print(f'Epoch: {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {val_loss:.4f}')

#     # Zapisywanie modelu, gdy jest najlepszy na walidacji
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         torch.save(model.state_dict(), 'best_model.pth')
#         print(f'Best model saved with validation loss: {val_loss:.4f}')


In [ ]:
# DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=15)

In [19]:
class CWT_EEG(LightningModule):
    def __init__(self,batch_size, input_size, hidden_size, num_layers, lr , label_smoothing=0):
        super().__init__()
        self.hparams.batch_size=batch_size
        self.hparams.lr = lr
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_of_classes = 3

        self.save_hyperparameters()
        self.val_percent = 0.2
        self.loss = torch.nn.CrossEntropyLoss(label_smoothing=label_smoothing)

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, self.num_of_classes)  # Klasyfikacja na 3 klasy

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, (hn, cn) = self.lstm(x, (h0, c0), device=x.device)
        out = hn[-1, :, :]  # Ostatnia warstwa LSTM (batch_size, hidden_size)
        out = self.fc(out)

        return out

    # custom
    def count_parameters(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

    # only for HP
    def on_train_start(self):
        self.logger.log_hyperparams(
            self.hparams,
            {
                "hp/train_loss": float("nan"),
                "hp/train_acc": float("nan"),
                "hp/val_loss": float("nan"),
                "hp/val_acc": float("nan"),
            },
        )

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        self.log("hp/train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("hp/train_acc", acc, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        self.log("hp/val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("hp/val_acc", acc, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def setup(self, stage=None):
        self.ds = CWTDataset("cwt_data.db", 4000)
        val_count = int(self.val_percent * len(self.ds))
        self.train_set, self.val_set = torch.utils.data.random_split(
            self.ds, [len(self.ds) - val_count, val_count]
        )

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_set, batch_size=self.hparams.batch_size, num_workers=12
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.val_set, batch_size=self.hparams.batch_size, num_workers=12
        )

In [20]:
lr = 0.001

model = CWT_EEG( batch_size= 4000 ,input_size=10 , num_layers=10,hidden_size=10, lr=lr)
logger = TensorBoardLogger("lightning_logs", name="CWT_EEG")
trainer = Trainer(
    max_epochs=10,
    logger=logger
)
trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
